In [1]:
import sbmlnetwork as sb
import pandas as pd

from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model
from math import log
from pathlib import Path


path = Path(r"C:\Users\mkcoo\Documents\UW\GitHub\PNSB\PettRidge\rpal.xml")
model = read_sbml_model(str(path.resolve()))

mdetails = pd.read_excel(r'C:\Users\mkcoo\Documents\UW\GitHub\PNSB\PettRidge\modelDetails.xlsx', sheet_name='Reactions')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C00234 "10-Formyltetrahydrofolate">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C15606 "1,2-dihydroxy-3-keto-5-methylthiopentene">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C00641DA "1,2-Diacyl-sn-glycerol (Dark Aerobic)">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C00641LN "1,2-Diacyl-sn-glycerol (Light Anaerobic)">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C04092 "1,2-Didehydropiperidine-2-carboxylate">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species C05758 "trans-Dodec-2-enoyl-[acp]">
Use of the spec

In [2]:
model.reactions.XR242.bounds = (1, 1) # coumarate
model.reactions.XR57.bounds = (0, 0) # acetate

# CO2 import/export bounds
model.reactions.XR72.bounds = (0, 0)
model.reactions.XR73.bounds = (0, 1000)
model.reactions.XR59.bounds = (0, 100)

In [3]:
ktokegg = {
    'J1': 'R00024',
    'J3': 'R01523',
    'J4': 'R01829',
    'J5': 'R01845',
    'J8': 'R00268',
    'J9': 'R00342',
    'J10': 'R00351',
    'J11': 'R00405',
    'J12': 'R00408',
    'J13': 'R00621',
    'J14': 'R01082',
    'J15': 'R01196',
    'J16': 'R01197',
    'J17': 'R01325',
    'J18': 'R01899',
    'J19': 'R01900',
    'J20': 'R02570',
    'J21': 'R03316',
    'J22': 'R04386',
    'J23': 'R00835',
    'J26': 'R01056',
    'J28': 'R01067',
    'J30': 'R01528',
    'J31': 'R01529',
    'J33': 'R01641',
    'J35': 'R01827',
    'J36': 'R02035',
    'J38': 'R00206',
    'J39': 'R00216',
    'J40': 'R00345',
    'J41': 'R00472',
    'J43': 'R01515',
    'J44': 'R01512',
    'J45': 'R01061',
    'J46': 'R01068',
    'J47': 'R00762',
    'J48': 'R02740',
    'J49': 'R01015',
    'J50': 'R00658',
    'J51': 'R01518',
    'J52': 'R00479'
}

In [4]:
solution = model.optimize()
stand_df = solution.to_frame()

In [5]:
flux_dict = {}

In [6]:
for j in ktokegg.keys():
    k = ktokegg[j]
    flux_dict[j] = abs(stand_df.loc[k]['fluxes'])

In [7]:
# Define an Antimony model
antimony_str = """
    // Reactions
    J1: rubp + h2o + co2 -> 2 h + 2 _3pg; k1*h2o
    J3: ru5pD + atp -> adp + h + rubp; k1*atp
    J4: sdp -> dhap + e4p; k1*sdp
    J5: sdp + h2o -> p + s7p; k1*sdp
    J8: oxsucc + h -> co2 + akg; k1*h
    J9: malL + nad -> nadh + h + oaa; k1*nad
    J10: oaa + accoa + h2o -> h + coa + cit; k1*h2o
    J11: succoa + adp + p -> coa + atp + succ; k1*atp
    J12: succ + fad -> fadh2 + fum; k1*fad
    J13: akg + thmpp -> co2 + _3car1hy; k1*akg
    J14: fum + h2o -> malL; k1*malL
    J15: pyr + coa + 2 oxferr -> h + rferr + co2 + accoa; k1*co2
    J16: akg + coa + 2 oxferr -> h + 2 rferr + co2 + succoa; k1*co2
    J17: cit -> h2o + cacon; k1*cit
    J18: icit + nadp -> nadph + h + oxsucc; k1*nadp
    J19: cacon + h2o -> icit; k1*icit
    J20: succoa + dhlp + h -> coa + sucdhl; k1*succoa
    J21: _3car1hy + lp + 2 h -> thmpp + sucdhl; k1*lp
    J22: accoa + CoBcoCP + h ->  BcoCP + malcoa; k1*accoa
    J23: g6p + nadp -> nadph + h + _6pgl; k1*nadp
    J26: ru5pD -> r5p; k1*r5p
    J28: f6p + g3p -> xu5pD + e4p; k1*f6p
    J30: _6pgc + nadp -> nadph + co2 + ru5pD; k1*nadp
    J31: ru5pD -> xu5pD; k1*ru5pD
    J33: g3p + s7p -> r5p + xu5pD; k1*g3p
    J35: g3p + s7p -> f6p + e4p; k1*g3p
    J36: _6pgl + h2o -> h + _6pgc; k1*h2o
    J38: pep + amp + h + pp -> p + atp + pyr; k1*atp
    J39: malL + nadp -> co2 + nadph + pyr; k1*nadp
    J40: oaa + p + h -> h2o + co2 + pep; k1*p
    J41: glx + h2o + accoa -> coa + h + malL; k1*h2o
    J43: _13dpg + h2o -> p + h + _3pg; k1*h2o
    J44: _3pg + atp -> adp + _13dpg; k1*_3pg
    J45: _13dpg + nadh + h -> p + nad + g3p; k1*g3p
    J46: g3p + dhap -> fdp; k1*fdp
    J47: fdp + h2o => p + f6p; k1*fdp
    J48: f6p -> g6p; k1*g6p
    J49: g3p -> dhap; k1*g3p
    J50: pep + h2o -> _2pg; k1*pep
    J51: _2pg -> _3pg; k1*_2pg
    J52: icit -> succ + glx; k1*icit

    // Species initializations
    _2pg = 1
    hv = 1
    pyr = 1
    cacon = 1
    icit = 1
    h2o = 1
    co2 = 1
    rubp = 1
    h = 1
    _3pg = 1
    nadp = 1
    nadph = 1
    atp = 1
    ru5pD = 1
    adp = 1
    sdp = 1
    dhap = 1
    e4p = 1
    s7p = 1
    p = 1
    cmono = 1
    q8 = 1
    focytc = 1
    q8h2 = 1
    ficytc = 1
    oxsucc = 1
    akg = 1
    nad = 1
    malL = 1
    nadh = 1
    oaa = 1
    accoa = 1
    coa = 1
    cit = 1
    succ = 1
    succoa = 1
    fad = 1
    fum = 1
    fadh2 = 1
    thmpp = 1
    _3car1hy = 1
    rferr = 1
    oxferr = 1
    dhlp = 1
    sucdhl = 1
    lp = 1
    CoBcoCP = 1
    malcoa = 1
    BcoCP = 1
    g6p = 1
    _6pgl = 1
    r5p = 1
    amp = 1
    prpp = 1
    ribD = 1
    r1p = 1
    f6p = 1
    g3p = 1
    xu5pD = 1
    gllcn = 1
    glcn = 1
    _6pgc = 1
    actp = 1
    _2dr5p = 1
    deoxyr = 1
    pp = 1
    pep = 1
    glx = 1
    _13dpg = 1

    // Parameter initialization
    k1 = 1;
"""

# Load the model from the Antimony string
net = sb.load(antimony_str)

In [8]:
net.set_font_color('#000080')

In [9]:
species_list = net.get_species_list()
species_list.set_shapes("circle")

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [10]:
core_species = net.get_species_list([
    "rubp", "_3pg", "ru5pd", "sdp", "e4p", "s7p", "oxsucc", "akg", "malL", "oaa", "accoa", "cit", "succ", "succoa", "fum",
    "_3car1hy", "pyr", "cacon", "icit", "sucdhl", "malcoa", "g6p", "_6pgl", "r5p", "prpp", "ribD", "ru5pD", "r1p", "f6p", "g3p", "xu5pD",
    "e4p", "gllcn", "glcn", "_6pgc", "_2dr5p", "deoxyr", "pep", "glx", "_13dpg", "dhap", "_2pg", "fdp"
])

In [11]:
cofactors = net.get_species_list([
    "hv", "h2o", "h", "nadh", "nad", "nadp", "nadph", "p", "ppi", "atp", "adp", "pp", "actp", "amp", "fad", "fadh2",
    "q8", "focytc", "q8h2", "ficytc", "rferr", "oxferr", "co2", 'BcoCP', 'CoBcoCP', 'coa', 'dhlp', 'thmpp'
])

In [12]:
net.create_aliases({
    'J1': ['h2o', 'co2', 'h'],
    'J3': ['atp', 'adp', 'h'],
    'J5': ['h2o', 'p'],
    'J8': ['h', 'co2'],
    'J9': ['nad', 'nadh', 'h'],
    'J10': ['h2o', 'h', 'coa'],
    'J11': ['atp', 'adp', 'p', 'coa'],
    'J12': ['fad', 'fadh2'],
    'J13': ['thmpp', 'co2'],
    'J14': ['h2o'],
    'J15': ['co2', 'rferr', 'oxferr', 'h', 'coa'],
    'J16': ['co2', 'rferr', 'oxferr', 'h', 'coa'],
    'J17': ['h2o'],
    'J18': ['nadp', 'nadph', 'h'],
    'J19': ['h2o'],
    'J20': ['h', 'coa', 'dhlp'],
    'J21': ['h', 'lp', 'thmpp'],
    'J22': ['h', 'CoBcoCP', 'BcoCP'],
    'J23': ['nadp', 'nadph', 'h'],
    'J30': ['nadp', 'nadph', 'co2'],
    'J36': ['h2o', 'h'],
    'J38': ['atp', 'amp', 'h', 'pp', 'p'],
    'J39': ['nadp', 'nadph', 'co2'],
    'J40': ['p', 'h', 'h2o', 'co2'],
    'J41': ['h2o', 'accoa', 'coa', 'h'],
    'J43': ['h2o', 'p', 'h'],
    'J44': ['atp', 'adp'],
    'J45': ['nad', 'nadh', 'p', 'h'],
    'J47': ['h2o', 'p'],
    'J50': ['h2o']
})

In [13]:
compartment = net.get_compartment()
# compartment.set_font_color('#000080')

In [14]:
species_list.set_font_sizes(60)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [15]:
core_species.set_sizes((200, 200))
cofactors.set_sizes((100, 100))

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [16]:
reactions = net.get_reactions_list()
reactions.switch_to_curve()
reactions.set_thicknesses(22)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [17]:
net.get_reaction("J31").get_species_list()[1].set_position((4300, 2500)) #x5p
net.get_reaction("J26").get_species_list()[1].set_position((5300, 2500)) #r5p
net.get_reaction("J33").get_species_list()[1].set_position((4300, 2000)) #s7p
net.get_reaction("J33").get_species_list()[0].set_position((5300, 2000)) #g3p
net.get_reaction("J5").get_species_list()[0].set_position((4300, 1500)) #sdp
net.get_reaction("J4").get_species_list()[1].set_position((4300, 1100)) #dhap
net.get_reaction("J4").get_species_list()[2].set_position((5000, 1000)) #e4p
net.get_reaction('J41').get_species_list()[0].set_position((2100, 5900)) #glx
net.get_reaction('J41').get_species_list()[2].set_position((2000.0, 5000.0)) #accoa
net.get_reaction('J22').get_species_list()[4].set_position((3700, 3910.0)) #malcoa

True

In [18]:
net.get_species_list('accoa')[0].get_position()

(7825.71, 4429.45)

In [19]:
net.get_reaction("J31").set_position((4811.6, 2944.9))
net.get_reaction("J26").set_position((5300, 2944.9))
net.get_reaction("J33").set_position((4800, 2250))
net.get_reaction("J5").set_position((4300, 1750))
net.get_reaction("J4").set_position((4600, 1300))
net.get_reaction("J52").set_position((2255, 6382.73))
net.get_reaction("J41").set_position((1534.64, 5595.51))
# net.get_reaction("J52").set_position((2900, 6600))
# net.get_reaction('J42').set_position((1375, 4742))
net.get_reaction('J44').set_position((3549, 2388))
net.get_reaction('J39').set_position((969.28, 3470.0))
net.get_reaction('J40').set_position((1766.77, 3000.0))
net.get_reaction('J28').set_position((5000, 1500))
net.get_reaction('J35').set_position((5236, 1500))
net.get_reaction('J49').set_position((4034.6, 1045.5))
net.get_reaction('J51').set_position((3400, 2800))

net.get_reaction("J38").set_position((2125.0, 3000.0))
net.get_reaction("J15").set_position((2125.0, 3800.0))
net.get_reaction('J22').set_position((3000.0, 3800))

True

In [20]:
# net.get_reactions_list('J33')[0].assign_species(net.get_reactions_list('J31').get_species_list()[0])
net.get_reactions_list('J33')[0].assign_species(net.get_reactions_list('J26').get_species_list()[1])
net.get_reactions_list('J31')[0].assign_species(net.get_reactions_list('J26').get_species_list()[1])
net.get_reactions_list('J41')[0].assign_species(net.get_reactions_list('J14').get_species_list()[0])
net.get_reactions_list('J41')[0].assign_species(net.get_reactions_list('J14').get_species_list()[2])
net.get_reactions_list('J44')[0].assign_species(net.get_reactions_list('J43').get_species_list()[0])
net.get_reactions_list('J44')[0].assign_species(net.get_reactions_list('J43').get_species_list()[4])
net.get_reactions_list('J51')[0].assign_species(net.get_reactions_list('J1').get_species_list()[4])
net.get_reactions_list('J31')[0].assign_species(net.get_reactions_list('J3').get_species_list()[0])

True

In [21]:
net.get_species_list('accoa')[0].set_position((2025, 3910))
net.get_reaction("J15").get_curves_list()[7].get_segments()[0].set_start((2125, 3800.0))
net.get_reaction("J15").get_curves_list()[7].get_segments()[0].set_end((2125, 3910.0))
net.get_reaction('J15').get_curves_list()[7].get_segments().set_control_point_1s(((2125, 3850)))
net.get_reaction('J15').get_curves_list()[7].get_segments().set_control_point_2s(((2125, 3900)))

net.get_species_list('pep')[0].set_position((2025, 2700.0))
net.get_reaction("J38").get_curves_list()[0].get_segments()[0].set_start((2125, 3000.0))
net.get_reaction("J38").get_curves_list()[0].get_segments()[0].set_end((2125, 2830.0))
net.get_reaction('J38').get_curves_list()[0].get_segments().set_control_point_1s(((2125, 2850)))
net.get_reaction('J38').get_curves_list()[0].get_segments().set_control_point_2s(((2125, 2900)))

net.get_species_list('pyr')[0].set_position((2025, 3200.0))
net.get_reaction("J15").get_curves_list()[0].get_segments()[0].set_start((2125, 3800.0))
net.get_reaction("J15").get_curves_list()[0].get_segments()[0].set_end((2125, 3000.0))
net.get_reaction('J15').get_curves_list()[0].get_segments().set_control_point_1s(((2125, 3550)))
net.get_reaction('J15').get_curves_list()[0].get_segments().set_control_point_2s(((2125, 3200)))

net.get_reaction("J38").get_curves_list()[6].get_segments()[0].set_start((2125, 3000.0))
net.get_reaction("J38").get_curves_list()[6].get_segments()[0].set_end((2125, 3170.0))
net.get_reaction('J38').get_curves_list()[6].get_segments().set_control_point_1s(((2125, 3100)))
net.get_reaction('J38').get_curves_list()[6].get_segments().set_control_point_2s(((2125, 3150)))

[True]

In [22]:
net.get_reaction("J22").get_curves_list()[0].get_segments()[0].set_start((2215.0, 4010)) #accoa
net.get_reaction("J22").get_curves_list()[0].get_segments()[0].set_end((2600, 4010.0))
net.get_reaction('J22').get_curves_list()[0].get_segments().set_control_point_1s(((2700, 4010.0)))
net.get_reaction('J22').get_curves_list()[0].get_segments().set_control_point_2s(((2800.77, 4010.0)))

net.get_reaction("J22").get_curves_list()[4].get_segments()[0].set_start((2600.0, 4010.0)) #malcoa
net.get_reaction("J22").get_curves_list()[4].get_segments()[0].set_end((3700.0, 4010.0))
net.get_reaction('J22').get_curves_list()[4].get_segments().set_control_point_1s(((2700, 4010.0)))
net.get_reaction('J22').get_curves_list()[4].get_segments().set_control_point_2s(((2800.77, 4010.0)))

net.get_reaction('J22').get_species_list()[4].set_position((3700, 3910)) #malcoa

True

In [23]:
net.auto_layout()

In [24]:
net.get_reactions_list(['J47', 'J48', 'J23', 'J36', 'J30',
                        'J3', 'J1', 'J43', 'J45', 'J46'
                        ]).align_circle(
                            center_at=(5000, 2000), radius=1500,
                            arc_start=105, arc_end=-255
                        )

net.get_reactions_list(['J10', 'J17', 'J19', 'J18', 'J8', 'J16', 'J11', 'J12', 'J14', 'J9'
                        ]).align_circle(
                            center_at=(2200, 6000), radius=1500,
                            arc_start=105, arc_end=-255
                        )

# net.get_reaction("J22").align_horizontal(center_at=(2500, 4270.0), spread = 1000) #fa biosyn

net.get_reaction("J15").align_vertical(center_at=(2200.0, 3800), spread = 800) #accoa to pyr
net.get_reaction("J38").align_vertical(center_at=(2200.0, 3000), spread = 800) #pyr to pep
net.get_reaction("J50").align_horizontal(center_at=(2800, 2800), spread = 1200) #pep to 2pg
net.get_reaction("J51").align_horizontal(center_at=(3400, 2800), spread = 800) #2pg to 3pg

net.get_reaction("J21").align_horizontal(center_at=(2060.34, 8583.89), spread = 925.78)
net.get_reaction("J13").align_vertical(center_at=(2523.23, 7983.89), spread = 1200)
net.get_reaction("J20").align_vertical(center_at=(1597.45, 7959.63), spread = 1248.52)

True

In [25]:
# net.get_reactions_list('J33')[0].assign_species(net.get_reactions_list('J31').get_species_list()[0])
net.get_reactions_list('J33')[0].assign_species(net.get_reactions_list('J26').get_species_list()[1])
net.get_reactions_list('J31')[0].assign_species(net.get_reactions_list('J26').get_species_list()[1])
net.get_reactions_list('J41')[0].assign_species(net.get_reactions_list('J14').get_species_list()[0])
net.get_reactions_list('J41')[0].assign_species(net.get_reactions_list('J14').get_species_list()[2])
net.get_reactions_list('J44')[0].assign_species(net.get_reactions_list('J43').get_species_list()[0])
net.get_reactions_list('J44')[0].assign_species(net.get_reactions_list('J43').get_species_list()[4])
net.get_reactions_list('J51')[0].assign_species(net.get_reactions_list('J1').get_species_list()[4])
net.get_reactions_list('J31')[0].assign_species(net.get_reactions_list('J3').get_species_list()[0])

True

In [26]:
net.get_reaction("J4").get_species_list()[1].set_position((4300, 1100)) #dhap

True

In [27]:
net.create_aliases({
    'J41': ['h2o', 'accoa', 'coa', 'h'],
    'J40': ['co2'],
    'J39': ['nadp', 'nadph'],
    'J21': ['lp', 'thmpp'],
    'J44': ['atp'],
    'J5': ['h2o', 'p']
})

In [28]:
net.update_reactions_curves()

In [29]:
net.get_reactions_list(['J47', 'J48', 'J23', 'J36', 'J30',
                        'J3', 'J1', 'J43', 'J45', 'J46'
                        ]).align_circle(
                            center_at=(5000, 2000), radius=1500,
                            arc_start=105, arc_end=-255
                        )

net.get_reactions_list(['J10', 'J17', 'J19', 'J18', 'J8', 'J16', 'J11', 'J12', 'J14', 'J9'
                        ]).align_circle(
                            center_at=(2200, 6000), radius=1500,
                            arc_start=105, arc_end=-255
                        )

# net.get_reaction("J22").align_horizontal(center_at=(4135, 4270.0), spread = 4000) #fa biosyn

net.get_reaction("J15").align_vertical(center_at=(2100.0, 3800), spread = 800) #accoa to pyr
net.get_reaction("J38").align_vertical(center_at=(2100.0, 3000), spread = 800) #pyr to pep
net.get_reaction("J50").align_horizontal(center_at=(2800, 2800), spread = 1200) #pep to 2pg
net.get_reaction("J51").align_horizontal(center_at=(3400, 2800), spread = 800) #2pg to 3pg

net.get_reaction("J21").align_horizontal(center_at=(2060.34, 8583.89), spread = 925.78)
net.get_reaction("J13").align_vertical(center_at=(2523.23, 7983.89), spread = 1200)
net.get_reaction("J20").align_vertical(center_at=(1597.45, 7959.63), spread = 1248.52)

True

In [30]:
net.get_reaction("J4").get_species_list()[1].set_position((4300, 1100)) #dhap

True

In [31]:
net.get_reaction('J40').get_curves_list()[0].get_segments()[0].set_start((1766.77, 4486.11))
net.get_reaction('J40').get_curves_list()[0].get_segments()[0].set_end((1766.77, 2798.24))
net.get_reaction('J40').get_curves_list()[0].get_segments().set_control_point_1s(((1766.77, 3798.24)))
net.get_reaction('J40').get_curves_list()[0].get_segments().set_control_point_2s(((1766.77, 4098.24)))

net.get_reaction('J40').get_curves_list()[5].get_segments()[0].set_start((1766.77, 2798.24))
net.get_reaction('J40').get_curves_list()[5].get_segments()[0].set_end((2200.0, 2798.24))
net.get_reaction('J40').get_curves_list()[5].get_segments().set_control_point_1s(((1856.77, 2798.24)))
net.get_reaction('J40').get_curves_list()[5].get_segments().set_control_point_2s(((1956.77, 2798.24)))

net.get_reaction('J40').get_curves_list()[1].hide()
net.get_reaction('J40').get_curves_list()[2].hide()
net.get_reaction('J40').get_species_list()[1].hide()
net.get_reaction('J40').get_species_list()[2].hide()

True

In [32]:
net.get_reaction('J39').get_curves_list()[0].get_segments()[0].set_start((979.28, 4991.02))
net.get_reaction('J39').get_curves_list()[0].get_segments()[0].set_end((979.28, 3300))
net.get_reaction('J39').get_curves_list()[0].get_segments().set_control_point_1s(((979.28, 3980.0)))
net.get_reaction('J39').get_curves_list()[0].get_segments().set_control_point_2s(((979.28, 3880.0)))

net.get_reaction('J39').get_curves_list()[4].get_segments()[0].set_start((979.28, 3300))
net.get_reaction('J39').get_curves_list()[4].get_segments()[0].set_end((2125.28, 3300))
net.get_reaction('J39').get_curves_list()[4].get_segments().set_control_point_1s(((1856.77, 3300)))
net.get_reaction('J39').get_curves_list()[4].get_segments().set_control_point_2s(((1956.77, 3300)))

net.get_reaction('J39').get_curves_list()[1].get_segments()[0].set_start((979.28, 4000)) #nadp
net.get_reaction('J39').get_curves_list()[1].get_segments()[0].set_end((750, 4050))
net.get_reaction('J39').get_curves_list()[1].get_segments().set_control_point_1s(((979.28, 4050.0)))
net.get_reaction('J39').get_curves_list()[1].get_segments().set_control_point_2s(((979.28, 4050.0)))

net.get_reaction('J39').get_curves_list()[3].get_segments()[0].set_start((979.28, 4000)) #nadph
net.get_reaction('J39').get_curves_list()[3].get_segments()[0].set_end((750, 3950))
net.get_reaction('J39').get_curves_list()[3].get_segments().set_control_point_1s(((979.28, 3950)))
net.get_reaction('J39').get_curves_list()[3].get_segments().set_control_point_2s(((979.28, 3950)))

[True]

In [33]:
net.get_reaction('J39').get_species_list()[1].set_position((710, 4010))
net.get_reaction('J39').get_species_list()[3].set_position((710, 3910))

True

In [34]:
net.get_reaction('J46').get_curves_list()[1].get_segments()[0].set_start((4476.11, 594.46)) #nadp
net.get_reaction('J46').get_curves_list()[1].get_segments()[0].set_end((4400, 1100)) # smaller x, larger y
net.get_reaction('J46').get_curves_list()[1].get_segments().set_control_point_1s((4476.11, 1100))
net.get_reaction('J46').get_curves_list()[1].get_segments().set_control_point_2s((4476.11, 1100))

[True]

In [35]:
net.get_reaction('J43').get_curves_list()[0].get_segments()[0].set_start((3571.11, 2388.23)) #13dpg
net.get_reaction('J43').get_curves_list()[0].get_segments()[0].set_end((3680.71, 2691.8))
net.get_reaction('J43').get_curves_list()[0].get_segments().set_control_point_1s((3611.85, 2540.27))
net.get_reaction('J43').get_curves_list()[0].get_segments().set_control_point_2s((3676.27, 2684.95))

net.get_reaction('J43').get_curves_list()[4].get_segments()[0].set_start((3591.11, 2388.23)) #3pg
net.get_reaction('J43').get_curves_list()[4].get_segments()[0].set_end((3535.24, 2051.45))
net.get_reaction('J43').get_curves_list()[4].get_segments().set_control_point_1s((3550.37, 2236.19))
net.get_reaction('J43').get_curves_list()[4].get_segments().set_control_point_2s((3533.82, 2078.68))

net.get_reaction('J44').get_curves_list()[0].get_segments()[0].set_start((3539.0, 2398.0)) #3pg
net.get_reaction('J44').get_curves_list()[0].get_segments()[0].set_end((3550.11, 2383.42))
net.get_reaction('J44').get_curves_list()[0].get_segments().set_control_point_1s((3531.89, 2374.03))
net.get_reaction('J44').get_curves_list()[0].get_segments().set_control_point_2s((3536.78, 2349.55))

net.get_reaction('J44').get_curves_list()[3].get_segments()[0].set_start((3539.0, 2398.0)) #13dpg
net.get_reaction('J44').get_curves_list()[3].get_segments()[0].set_end((3483.58, 2030.87))
net.get_reaction('J44').get_curves_list()[3].get_segments().set_control_point_1s((3546.11, 2421.97))
net.get_reaction('J44').get_curves_list()[3].get_segments().set_control_point_2s((3485.86, 2194.94))

net.get_reaction('J44').get_curves_list()[1].get_segments()[0].set_start((3572.0, 2388.0)) #atp
net.get_reaction('J44').get_curves_list()[1].get_segments()[0].set_end((3750, 2488))
net.get_reaction('J44').get_curves_list()[1].get_segments().set_control_point_1s((3650.0, 2498))
net.get_reaction('J44').get_curves_list()[1].get_segments().set_control_point_2s((3650.0, 2498))

net.get_reaction('J44').get_curves_list()[2].get_segments()[0].set_start((3572.0, 2388.0)) #adp
net.get_reaction('J44').get_curves_list()[2].get_segments()[0].set_end((3750, 2288))
net.get_reaction('J44').get_curves_list()[2].get_segments().set_control_point_1s((3620.0, 2298))
net.get_reaction('J44').get_curves_list()[2].get_segments().set_control_point_2s((3620.86, 2298))

net.get_reaction('J44').get_species_list()[1].set_position((3750, 2478))
net.get_reaction('J44').get_species_list()[2].set_position((3750, 2270))

True

In [36]:
net.get_reaction("J51").get_curves_list()[0].get_segments()[0].set_start((3410.0, 2800.0))
net.get_reaction("J51").get_curves_list()[0].get_segments()[0].set_end((3209.87, 2800))
net.get_reaction('J51').get_curves_list()[0].get_segments().set_control_point_1s(((3360, 2800.0)))
net.get_reaction('J51').get_curves_list()[0].get_segments().set_control_point_2s(((3300.77, 2800.0)))

net.get_reaction("J51").get_curves_list()[1].get_segments()[0].set_start((3410.0, 2800.0))
net.get_reaction("J51").get_curves_list()[1].get_segments()[0].set_end((3600.0, 2800.0))
net.get_reaction('J51').get_curves_list()[1].get_segments().set_control_point_1s(((3560, 2800.0)))
net.get_reaction('J51').get_curves_list()[1].get_segments().set_control_point_2s(((3500.77, 2800.0)))

[True]

In [37]:
net.get_reaction("J50").get_curves_list()[0].get_segments()[0].set_start((2600.0, 2800.0))
net.get_reaction("J50").get_curves_list()[0].get_segments()[0].set_end((2125.0, 2800.0))
net.get_reaction('J50').get_curves_list()[0].get_segments().set_control_point_1s(((2501, 2800.0)))
net.get_reaction('J50').get_curves_list()[0].get_segments().set_control_point_2s(((2550.77, 2800.0)))

net.get_reaction("J50").get_curves_list()[2].get_segments()[0].set_start((2600.0, 2800.0))
net.get_reaction("J50").get_curves_list()[2].get_segments()[0].set_end((3000.0, 2800.0))
net.get_reaction('J50').get_curves_list()[2].get_segments().set_control_point_1s(((2700, 2800.0)))
net.get_reaction('J50').get_curves_list()[2].get_segments().set_control_point_2s(((2800.77, 2800.0)))

[True]

In [38]:
net.get_reaction("J22").get_curves_list()[0].get_segments()[0].set_start((2215.0, 4010)) #accoa
net.get_reaction("J22").get_curves_list()[0].get_segments()[0].set_end((2600, 4010.0))
net.get_reaction('J22').get_curves_list()[0].get_segments().set_control_point_1s(((2700, 4010.0)))
net.get_reaction('J22').get_curves_list()[0].get_segments().set_control_point_2s(((2800.77, 4010.0)))

net.get_reaction("J22").get_curves_list()[4].get_segments()[0].set_start((2600.0, 4010.0)) #malcoa
net.get_reaction("J22").get_curves_list()[4].get_segments()[0].set_end((3700.0, 4010.0))
net.get_reaction('J22').get_curves_list()[4].get_segments().set_control_point_1s(((2700, 4010.0)))
net.get_reaction('J22').get_curves_list()[4].get_segments().set_control_point_2s(((2800.77, 4010.0)))

net.get_reaction('J22').get_species_list()[4].set_position((3700, 3910)) #malcoa

True

In [39]:
net.get_reaction("J49").get_curves_list()[0].get_segments()[0].set_start((4017.14, 1028.01)) #g3p
net.get_reaction("J49").get_curves_list()[0].get_segments()[0].set_end((3943.97, 1055.2))
net.get_reaction('J49').get_curves_list()[0].get_segments().set_control_point_1s(((3943.97, 1055.2)))
net.get_reaction('J49').get_curves_list()[0].get_segments().set_control_point_2s(((4017.14, 1028.01)))

net.get_reaction("J49").get_curves_list()[1].get_segments()[0].set_start((4017.14, 1028.01)) #dhap
net.get_reaction("J49").get_curves_list()[1].get_segments()[0].set_end((4291.41, 1194.87))
net.get_reaction('J49').get_curves_list()[1].get_segments().set_control_point_1s(((4017.14, 1028.01)))
net.get_reaction('J49').get_curves_list()[1].get_segments().set_control_point_2s(((4291.41, 1194.87)))

[True]

In [40]:
net.get_species_list('accoa')[0].set_position((2025, 3910))
net.get_reaction("J15").get_curves_list()[7].get_segments()[0].set_start((2125, 3800.0))
net.get_reaction("J15").get_curves_list()[7].get_segments()[0].set_end((2125, 3910.0))
net.get_reaction('J15').get_curves_list()[7].get_segments().set_control_point_1s(((2125, 3850)))
net.get_reaction('J15').get_curves_list()[7].get_segments().set_control_point_2s(((2125, 3900)))

net.get_species_list('pep')[0].set_position((2025, 2700.0))
net.get_reaction("J38").get_curves_list()[0].get_segments()[0].set_start((2125, 3000.0))
net.get_reaction("J38").get_curves_list()[0].get_segments()[0].set_end((2125, 2830.0))
net.get_reaction('J38').get_curves_list()[0].get_segments().set_control_point_1s(((2125, 2850)))
net.get_reaction('J38').get_curves_list()[0].get_segments().set_control_point_2s(((2125, 2900)))

net.get_species_list('pyr')[0].set_position((2025, 3200.0))
net.get_reaction("J15").get_curves_list()[0].get_segments()[0].set_start((2125, 3800.0))
net.get_reaction("J15").get_curves_list()[0].get_segments()[0].set_end((2125, 3000.0))
net.get_reaction('J15').get_curves_list()[0].get_segments().set_control_point_1s(((2125, 3550)))
net.get_reaction('J15').get_curves_list()[0].get_segments().set_control_point_2s(((2125, 3200)))

net.get_reaction("J38").get_curves_list()[6].get_segments()[0].set_start((2125, 3000.0))
net.get_reaction("J38").get_curves_list()[6].get_segments()[0].set_end((2125, 3170.0))
net.get_reaction('J38').get_curves_list()[6].get_segments().set_control_point_1s(((2125, 3100)))
net.get_reaction('J38').get_curves_list()[6].get_segments().set_control_point_2s(((2125, 3150)))

[True]

In [41]:
for species in net.get_species_list('h'):
    species.hide(apply_to_connected_elements=True)

In [42]:
for species in net.get_species_list('BcoCP'):
    species.hide(apply_to_connected_elements=True)

In [43]:
for species in net.get_species_list('CoBcoCP'):
    species.hide(apply_to_connected_elements=True)

In [44]:
for species in net.get_species_list('p'):
    species.hide(apply_to_connected_elements=True)

In [45]:
for species in net.get_species_list('ppi'):
    species.hide(apply_to_connected_elements=True)

In [46]:
for species in net.get_species_list('h2o'):
    species.hide(apply_to_connected_elements=True)

In [47]:
for species in net.get_species_list('pp'):
    species.hide(apply_to_connected_elements=True)

In [48]:
net.show_fluxes(data=flux_dict, log_scale=False)

True

In [49]:
reactions.set_thicknesses(22)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [50]:
net.draw('Fig2B.pdf')

In [51]:
model.reactions.R01195.bounds = (0, 0)

In [52]:
solution = model.optimize()
stand_df = solution.to_frame()

In [53]:
flux_dict = {}

In [54]:
for j in ktokegg.keys():
    k = ktokegg[j]
    flux_dict[j] = abs(stand_df.loc[k]['fluxes'])

In [55]:
net.show_fluxes(data=flux_dict, log_scale=False)

True

In [56]:
reactions.set_thicknesses(22)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [57]:
net.draw('Fig2C.pdf')

In [58]:
model.reactions.R01195.bounds = (-1000, 1000)
model.reactions.XR13.bounds = (0, 0)

In [59]:
solution = model.optimize()
stand_df = solution.to_frame()

In [60]:
flux_dict = {}

In [61]:
for j in ktokegg.keys():
    k = ktokegg[j]
    flux_dict[j] = abs(stand_df.loc[k]['fluxes'])

In [62]:
net.show_fluxes(data=flux_dict, log_scale=False)

True

In [63]:
reactions.set_thicknesses(22)

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [64]:
net.draw('Fig2D.pdf')